<a href="https://colab.research.google.com/github/ashwani0506/Brain_Tumor_Detection/blob/main/brain_tumor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import os

# Mount Google Drive
drive.mount("/content/drive", force_remount=True)

# Paths to dataset
train_dir = '/content/drive/My Drive/brain_tumor_dataset/brain_tumor_dataset/train/'
validation_dir = '/content/drive/My Drive/brain_tumor_dataset/brain_tumor_dataset/validation/'
test_dir = '/content/drive/My Drive/brain_tumor_dataset/brain_tumor_dataset/test/'

#DATA REPROCESSING
# Load training and validation datasets

train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32
)
validation_dataset = image_dataset_from_directory(
    validation_dir,
    image_size=(224, 224),
    batch_size=32
)
test_dataset = image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32
)

# Extract class names before prefetching
class_names = train_dataset.class_names
print(f"Class Names: {class_names}")

# Apply prefetching for optimization
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)




Mounted at /content/drive
Found 3431 files belonging to 4 classes.
Found 1030 files belonging to 4 classes.
Found 578 files belonging to 4 classes.
Class Names: ['glioma', 'meningioma', 'notumor', 'pituitary']


In [ ]:
#BUILDING THE MODEL

# Load the MobileNetV2 model (pre-trained on ImageNet) for transfer learning
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

# Build the custom model
# Build the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Replaces Flatten for efficiency
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')  # Use softmax for multi-class classification
])

# Compile the model
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


In [ ]:
# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=1
)


108/108 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - accuracy: 0.6699 - loss: 0.8441 - val_accuracy: 0.7738 - val_loss: 0.4782


In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_accuracy:.2f}")


19/19 ━━━━━━━━━━━━━━━━━━━━ 26s 1s/step - accuracy: 0.8475 - loss: 0.4694
Test Accuracy: 0.83


In [ ]:
# Save the trained model to Google Drive
model.save('/content/drive/MyDrive/brain_tumor_model_mobilenetv2.h5')
print("Model saved successfully!")


Model saved successfully!


In [ ]:
# Predict function for a single image
def predict_image(image_path):
    img = tf.keras.utils.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, axis=0) / 255.0  # Normalize image
    predictions = model.predict(img_array)
    predicted_class = class_names[tf.argmax(predictions[0])]
    return predicted_class

# Example: Predicting on a new test image
test_image_path = '/content/drive/My Drive/brain_tumor_dataset/brain_tumor_dataset/test/notumor/test_img.jpg'  # Replace with your test image path
predicted_class = predict_image(test_image_path)
print(f"Predicted Class: {predicted_class} ")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Class: notumor 
